# 성능튜닝

## 1.환경준비

### (1) import

In [1]:
#라이브러리들을 불러오자.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 전처리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 모델링
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import * 

import warnings    # 경고메시지 제외
warnings.filterwarnings(action='ignore')

### (2) 데이터 준비

* 변수설명
    * COLLEGE : 대학 졸업여부
    * INCOME : 연수입
    * OVERAGE : 월평균 초과사용 시간(분)
    * LEFTOVER : 월평균 잔여시간비율(%)
    * HOUSE : 집값
    * HANDSET_PRICE : 스마트폰 가격
    * OVER_15MINS_CALLS_PER_MONTH : 월평균 장기통화(15분이상) 횟수
    * AVERAGE_CALL_DURATION : 평균 통화 시간
    * REPORTED_SATISFACTION : 만족도 설문조사 결과
    * REPORTED_USAGE_LEVEL : 사용도 자가진단 결과
    * CONSIDERING_CHANGE_OF_PLAN : 향후 변경계획 설문조사 결과
    * CHURN : 이탈(번호이동) 여부 (1-이탈, 0-잔류, Target 변수)


In [2]:
# 데이터를 불러옵시다.
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/mobile_cust_churn.csv'
data = pd.read_csv(path)
data = data.sample(5000, random_state = 2022)
data['CHURN'] = data['CHURN'].map({'LEAVE':1, 'STAY':0})
# sklearn 에서는 y를 가변수화 할 필요 없다.
# statsmodels, 에서는 반드시 y를 가변수화 해야한다.
data.head()

,id,COLLEGE,INCOME,OVERAGE,LEFTOVER,HOUSE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION,REPORTED_SATISFACTION,REPORTED_USAGE_LEVEL,CONSIDERING_CHANGE_OF_PLAN,CHURN
3178,3179,0,119512,51,31,248566,229,5,2,very_sat,very_high,considering,1
14926,14927,1,142144,192,15,774317,581,29,4,unsat,very_little,never_thought,1
15116,15117,1,142308,0,79,306426,497,1,1,sat,little,considering,0
12733,12734,1,113385,0,0,333599,819,1,6,very_unsat,very_high,considering,1
14032,14033,1,90348,209,10,637286,360,26,4,unsat,little,actively_looking_into_it,0


## 2.데이터 준비

### (1) 데이터 정리

In [3]:
drop_cols = ['id']
data.drop(drop_cols, axis = 1, inplace = True )

### (2) 데이터분할1 : x, y 나누기

In [4]:
target = 'CHURN'
x = data.drop(target, axis = 1)
y = data.loc[:, target]

### (3) NA 조치

### (4) 가변수화

In [5]:
dumm_cols = ['REPORTED_SATISFACTION','REPORTED_USAGE_LEVEL','CONSIDERING_CHANGE_OF_PLAN']
x = pd.get_dummies(x, columns = dumm_cols, drop_first = True)

### (5) 데이터분할2 : train : validation 나누기

In [6]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .3, random_state = 20)

### (6) Scaling

In [7]:
scaler = MinMaxScaler()
x_train_s = scaler.fit_transform(x_train)
x_val_s = scaler.transform(x_val)

## 3.선형모델 튜닝

Logistic Regression : 전진선택법
* 변수를 하나씩 늘려가면서
* AIC를 가장 낮추는 모델 찾기

### (1) 전진선택을 수행할 함수 만들기( **로지스틱 회귀** 용)

In [8]:
# 아래 함수는 로지스틱 회귀를 위한 전진선택법 함수 입니다.
import statsmodels.api as sm

def forward_stepwise_logistic(x_train, y_train):

    # 변수목록, 선택된 변수 목록, 단계별 모델과 AIC 저장소 정의
    features = list(x_train)
    selected = []
    step_df = pd.DataFrame({ 'step':[], 'feature':[],'aic':[]})

    # 
    for s in range(0, len(features)) :
        result =  { 'step':[], 'feature':[],'aic':[]}

        # 변수 목록에서 변수 한개씩 뽑아서 모델에 추가
        for f in features :
            vars = selected + [f]
            x_tr = x_train[vars]
            model = sm.Logit(y_train, x_tr).fit()
            result['step'].append(s+1)
            result['feature'].append(vars)
            result['aic'].append(model.aic)
        
        # 모델별 aic 집계
        temp = pd.DataFrame(result).sort_values('aic').reset_index(drop = True)

        # 만약 이전 aic보다 새로운 aic 가 크다면 멈추기
        if step_df['aic'].min() < temp['aic'].min() :
            break
        step_df = pd.concat([step_df, temp], axis = 0).reset_index(drop = True)

        # 선택된 변수 제거
        v = temp.loc[0,'feature'][s]
        features.remove(v)

        selected.append(v)
    
    # 선택된 변수와 step_df 결과 반환
    return selected, step_df

### (2) 전진선택법 수행

In [9]:
vars, result = forward_stepwise_logistic(x_train, y_train)

Optimization terminated successfully.
         Current function value: 0.693075
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693062
         Iterations 2
Optimization terminated successfully.
         Current function value: 0.683528
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693007
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.684909
         Iterations 2
Optimization terminated successfully.
         Current function value: 0.693042
         Iterations 2
Optimization terminated successfully.
         Current function value: 0.686899
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692565
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692806
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693137
  

* 선택된 변수

In [10]:
vars

['OVERAGE',
 'HOUSE',
 'HANDSET_PRICE',
 'LEFTOVER',
 'REPORTED_SATISFACTION_very_sat',
 'INCOME',
 'REPORTED_SATISFACTION_sat']

In [11]:
list(x_train)

['COLLEGE',
 'INCOME',
 'OVERAGE',
 'LEFTOVER',
 'HOUSE',
 'HANDSET_PRICE',
 'OVER_15MINS_CALLS_PER_MONTH',
 'AVERAGE_CALL_DURATION',
 'REPORTED_SATISFACTION_sat',
 'REPORTED_SATISFACTION_unsat',
 'REPORTED_SATISFACTION_very_sat',
 'REPORTED_SATISFACTION_very_unsat',
 'REPORTED_USAGE_LEVEL_high',
 'REPORTED_USAGE_LEVEL_little',
 'REPORTED_USAGE_LEVEL_very_high',
 'REPORTED_USAGE_LEVEL_very_little',
 'CONSIDERING_CHANGE_OF_PLAN_considering',
 'CONSIDERING_CHANGE_OF_PLAN_never_thought',
 'CONSIDERING_CHANGE_OF_PLAN_no',
 'CONSIDERING_CHANGE_OF_PLAN_perhaps']

In [12]:
result

,step,feature,aic
0,1.0,[OVERAGE],4786.699456
1,1.0,[HOUSE],4796.363859
2,1.0,[OVER_15MINS_CALLS_PER_MONTH],4810.294604
3,1.0,[REPORTED_SATISFACTION_very_sat],4845.064834
4,1.0,[AVERAGE_CALL_DURATION],4849.951663
...,...,...,...
114,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPO...",4442.309949
115,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPO...",4442.315315
116,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPO...",4442.316816
117,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPO...",4442.448187


In [13]:
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [14]:
result

,step,feature,aic
0,1.0,[OVERAGE],4786.699456
1,1.0,[HOUSE],4796.363859
2,1.0,[OVER_15MINS_CALLS_PER_MONTH],4810.294604
3,1.0,[REPORTED_SATISFACTION_very_sat],4845.064834
4,1.0,[AVERAGE_CALL_DURATION],4849.951663
...,...,...,...
114,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPORTED_SATISFACTION_very_sat, INCOME, REPORTED_SATISFACTION_very_unsat]",4442.309949
115,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPORTED_SATISFACTION_very_sat, INCOME, CONSIDERING_CHANGE_OF_PLAN_never_thought]",4442.315315
116,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPORTED_SATISFACTION_very_sat, INCOME, REPORTED_USAGE_LEVEL_high]",4442.316816
117,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPORTED_SATISFACTION_very_sat, INCOME, REPORTED_USAGE_LEVEL_very_little]",4442.448187


### (3) 모델링

* 전체 변수 

In [15]:
m1 = LogisticRegression()
m1.fit(x_train, y_train)
p1 = m1.predict(x_val)

print(accuracy_score(y_val, p1))
print(classification_report(y_val, p1))

0.6333333333333333
              precision    recall  f1-score   support

           0       0.62      0.68      0.65       738
           1       0.65      0.59      0.62       762

    accuracy                           0.63      1500
   macro avg       0.63      0.63      0.63      1500
weighted avg       0.64      0.63      0.63      1500



* 전진선택법 변수

In [16]:
m2 = LogisticRegression()
m2.fit(x_train[vars], y_train)
p2 = m2.predict(x_val[vars])

print(accuracy_score(y_val, p2))
print(classification_report(y_val, p2))

0.634
              precision    recall  f1-score   support

           0       0.62      0.68      0.65       738
           1       0.66      0.59      0.62       762

    accuracy                           0.63      1500
   macro avg       0.64      0.63      0.63      1500
weighted avg       0.64      0.63      0.63      1500



## 4.하이퍼파라미터 튜닝

### (1) 필요한 함수 불러오기 

In [17]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

### (2) Random Search

① 값의 범위를 지정한다.  
② 모델 선언(시도 횟수 지정)  
③ 모델링(값의 범위 내에서 시도 횟수만큼 랜덤하게 선택해서 시도한다.)  
④ 가장 성능이 좋은 값을 선정


#### ① 값의 범위를 지정한다.

In [18]:
# dictionary형태로 선언
params = { 'n_neighbors' : range(1,51), 'metric' : ['euclidean', 'manhattan']  }
params

{'n_neighbors': range(1, 51), 'metric': ['euclidean', 'manhattan']}

#### ② 모델 선언

In [19]:
# 기본모델
model = KNeighborsClassifier()

# Random Search 설정.
model_rs = RandomizedSearchCV(model
                            , params              # hyperparameter 범위 지정.
                            , cv=5                    # k-fold Cross Validation
                            , n_iter=5                # Random하게 시도할 횟수
                            )

#### ③ 모델링

In [20]:
# 학습 : model이 아니라 model_rs
model_rs.fit(x_train_s, y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_iter=5,
                   param_distributions={'metric': ['euclidean', 'manhattan'],
                                        'n_neighbors': range(1, 51)})

In [21]:
# 튜닝 결과
model_rs.cv_results_

{'mean_fit_time': array([0.00156198, 0.00097642, 0.00117145, 0.00117126, 0.00097623]),
 'std_fit_time': array([1.17118371e-03, 1.29362678e-06, 9.56044050e-04, 3.90101368e-04,
        1.19304619e-06]),
 'mean_score_time': array([0.11280994, 0.08132038, 0.07918496, 0.07834258, 0.08063254]),
 'std_score_time': array([0.05024431, 0.00264256, 0.00365899, 0.00211762, 0.01394291]),
 'param_n_neighbors': masked_array(data=[38, 36, 29, 11, 44],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_metric': masked_array(data=['manhattan', 'manhattan', 'manhattan', 'manhattan',
                    'euclidean'],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_neighbors': 38, 'metric': 'manhattan'},
  {'n_neighbors': 36, 'metric': 'manhattan'},
  {'n_neighbors': 29, 'metric': 'manhattan'},
  {'n_neighbors': 11, 'metric': 'manhattan'},
  {'n_neighbors': 44, 'metri

In [22]:
model_rs.cv_results_['params']

[{'n_neighbors': 38, 'metric': 'manhattan'},
 {'n_neighbors': 36, 'metric': 'manhattan'},
 {'n_neighbors': 29, 'metric': 'manhattan'},
 {'n_neighbors': 11, 'metric': 'manhattan'},
 {'n_neighbors': 44, 'metric': 'euclidean'}]

In [23]:
model_rs.cv_results_['mean_test_score']

array([0.62771429, 0.62228571, 0.618     , 0.59828571, 0.59171429])

In [24]:
# 최적의 파라미터
model_rs.best_params_

{'n_neighbors': 38, 'metric': 'manhattan'}

In [25]:
# 그때의 성능
model_rs.best_score_

0.6277142857142857

In [26]:
# best 모델로 예측 및 평가
pred = model_rs.predict(x_val_s)
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.58      0.78      0.67       738
           1       0.68      0.46      0.55       762

    accuracy                           0.62      1500
   macro avg       0.63      0.62      0.61      1500
weighted avg       0.63      0.62      0.61      1500



### (3) 실습 : Random Search

* decision tree로 튜닝을 시도해 봅시다.
    * max_depth : 1~10
    * min_samples_leaf : 10 ~ 100

In [27]:
from sklearn.tree import DecisionTreeClassifier

#### ① 값의 범위를 지정한다.

In [28]:
# dictionary형태로 선언
params = { 'max_depth' : range(1, 11), 'min_samples_leaf' : range(10, 101)  }
params

{'max_depth': range(1, 11), 'min_samples_leaf': range(10, 101)}

#### ② 모델 선언

In [29]:
model = DecisionTreeClassifier()

model_rs = RandomizedSearchCV(model
                            , params              # hyperparameter 범위 지정.
                            , cv=5                    # k-fold Cross Validation
                            , n_iter=10               # Random하게 시도할 횟수
                            )

#### ③ 모델링

In [30]:
# 학습 : model이 아니라 model_rs
model_rs.fit(x_train_s, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(),
                   param_distributions={'max_depth': range(1, 11),
                                        'min_samples_leaf': range(10, 101)})

In [31]:
# 튜닝 결과
model_rs.cv_results_

{'mean_fit_time': array([0.00741758, 0.01249256, 0.00292835, 0.01093097, 0.00624671,
        0.00936942, 0.00839367, 0.00917449, 0.00409794, 0.00487995]),
 'std_fit_time': array([0.00181053, 0.00143521, 0.00061741, 0.00038984, 0.0009957 ,
        0.00266242, 0.00117104, 0.00047702, 0.00072979, 0.00106973]),
 'mean_score_time': array([0.00058532, 0.00097642, 0.00058537, 0.0011713 , 0.00117073,
        0.00078106, 0.00019507, 0.00039063, 0.00117259, 0.00117102]),
 'std_score_time': array([0.00047791, 0.00061748, 0.00047795, 0.00095666, 0.00073008,
        0.00039053, 0.00039015, 0.00047842, 0.00039075, 0.00073026]),
 'param_min_samples_leaf': masked_array(data=[97, 31, 34, 20, 81, 87, 97, 69, 43, 97],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[3, 9, 1, 7, 3, 9, 6, 6, 2, 1],
              mask=[False, False, False, False, False, Fal

In [32]:
model_rs.cv_results_['params']

[{'min_samples_leaf': 97, 'max_depth': 3},
 {'min_samples_leaf': 31, 'max_depth': 9},
 {'min_samples_leaf': 34, 'max_depth': 1},
 {'min_samples_leaf': 20, 'max_depth': 7},
 {'min_samples_leaf': 81, 'max_depth': 3},
 {'min_samples_leaf': 87, 'max_depth': 9},
 {'min_samples_leaf': 97, 'max_depth': 6},
 {'min_samples_leaf': 69, 'max_depth': 6},
 {'min_samples_leaf': 43, 'max_depth': 2},
 {'min_samples_leaf': 97, 'max_depth': 1}]

In [33]:
model_rs.cv_results_['mean_test_score']

array([0.69314286, 0.69742857, 0.614     , 0.68828571, 0.69314286,
       0.69142857, 0.69228571, 0.69057143, 0.66142857, 0.614     ])

In [34]:
# 최적의 파라미터
model_rs.best_params_

{'min_samples_leaf': 31, 'max_depth': 9}

In [35]:
# 그때의 성능
model_rs.best_score_

0.6974285714285713

In [36]:
# best 모델로 예측 및 평가
pred = model_rs.predict(x_val_s)
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.67      0.68      0.68       738
           1       0.69      0.68      0.68       762

    accuracy                           0.68      1500
   macro avg       0.68      0.68      0.68      1500
weighted avg       0.68      0.68      0.68      1500



### (4) Grid Search

① 값의 범위를 지정한다.  
② 모델링(값의 범위 내에서 모든 조합을 다 시도한다.)  
③ 가장 성능이 좋은 값을 선정


In [37]:
from sklearn.model_selection import GridSearchCV

#### ① 값의 범위를 지정한다.

In [38]:
# dictionary형태로 선언
params = { 'n_neighbors' : range(3,31,2), 'metric' : ['euclidean', 'manhattan']  }
params

{'n_neighbors': range(3, 31, 2), 'metric': ['euclidean', 'manhattan']}

#### ② 모델 선언

In [39]:
# 기본모델
model = KNeighborsClassifier()

# Random Search 설정.
model_gs = GridSearchCV(model, params, cv=5, verbose=3)

#### ③ 모델링

In [40]:
# 학습 : model이 아니라 model_rs
model_gs.fit(x_train_s, y_train)

Fitting 5 folds for each of 28 candidates, totalling 140 fits
[CV 1/5] END ...metric=euclidean, n_neighbors=3;, score=0.583 total time=   0.0s
[CV 2/5] END ...metric=euclidean, n_neighbors=3;, score=0.577 total time=   0.0s
[CV 3/5] END ...metric=euclidean, n_neighbors=3;, score=0.554 total time=   0.0s
[CV 4/5] END ...metric=euclidean, n_neighbors=3;, score=0.600 total time=   0.0s
[CV 5/5] END ...metric=euclidean, n_neighbors=3;, score=0.580 total time=   0.0s
[CV 1/5] END ...metric=euclidean, n_neighbors=5;, score=0.579 total time=   0.0s
[CV 2/5] END ...metric=euclidean, n_neighbors=5;, score=0.599 total time=   0.0s
[CV 3/5] END ...metric=euclidean, n_neighbors=5;, score=0.569 total time=   0.0s
[CV 4/5] END ...metric=euclidean, n_neighbors=5;, score=0.579 total time=   0.0s
[CV 5/5] END ...metric=euclidean, n_neighbors=5;, score=0.573 total time=   0.0s
[CV 1/5] END ...metric=euclidean, n_neighbors=7;, score=0.581 total time=   0.0s
[CV 2/5] END ...metric=euclidean, n_neighbors=7

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': range(3, 31, 2)},
             verbose=3)

In [ ]:
# 튜닝 결과
model_gs.cv_results_

{'mean_fit_time': array([0.00119681, 0.00119414, 0.00140429, 0.00109844, 0.0007978 ,
        0.00079541, 0.00129905, 0.00099826, 0.0009975 , 0.00111804,
        0.00139675, 0.00079789, 0.00099683, 0.00099716, 0.00099778,
        0.00060501, 0.00099711, 0.00080161, 0.001196  , 0.00118318,
        0.0009974 , 0.0012136 , 0.00039883, 0.00059333, 0.0009973 ,
        0.001197  , 0.0008039 , 0.00119128]),
 'std_fit_time': array([3.98779044e-04, 4.00632173e-04, 2.03037128e-04, 2.02512769e-04,
        3.98898221e-04, 3.97735399e-04, 5.95102604e-04, 2.33995416e-05,
        6.97552626e-07, 6.75672728e-04, 4.88636574e-04, 3.98945905e-04,
        9.88790577e-07, 5.95569420e-07, 3.01578299e-07, 4.94122183e-04,
        1.09153015e-06, 4.00857359e-04, 3.99307251e-04, 3.92591781e-04,
        1.90734863e-07, 3.91485409e-04, 4.88460925e-04, 4.84551437e-04,
        6.21719590e-07, 3.99044776e-04, 4.02114273e-04, 4.02150749e-04]),
 'mean_score_time': array([0.0546535 , 0.06509662, 0.06169186, 0.06239495, 

In [58]:
model_gs.cv_results_['params']

[{'metric': 'euclidean', 'n_neighbors': 3},
 {'metric': 'euclidean', 'n_neighbors': 5},
 {'metric': 'euclidean', 'n_neighbors': 7},
 {'metric': 'euclidean', 'n_neighbors': 9},
 {'metric': 'euclidean', 'n_neighbors': 11},
 {'metric': 'euclidean', 'n_neighbors': 13},
 {'metric': 'euclidean', 'n_neighbors': 15},
 {'metric': 'euclidean', 'n_neighbors': 17},
 {'metric': 'euclidean', 'n_neighbors': 19},
 {'metric': 'euclidean', 'n_neighbors': 21},
 {'metric': 'euclidean', 'n_neighbors': 23},
 {'metric': 'euclidean', 'n_neighbors': 25},
 {'metric': 'euclidean', 'n_neighbors': 27},
 {'metric': 'euclidean', 'n_neighbors': 29},
 {'metric': 'manhattan', 'n_neighbors': 3},
 {'metric': 'manhattan', 'n_neighbors': 5},
 {'metric': 'manhattan', 'n_neighbors': 7},
 {'metric': 'manhattan', 'n_neighbors': 9},
 {'metric': 'manhattan', 'n_neighbors': 11},
 {'metric': 'manhattan', 'n_neighbors': 13},
 {'metric': 'manhattan', 'n_neighbors': 15},
 {'metric': 'manhattan', 'n_neighbors': 17},
 {'metric': 'manha

In [59]:
model_gs.cv_results_['mean_test_score']

array([0.57885714, 0.57942857, 0.57714286, 0.57342857, 0.568     ,
       0.57457143, 0.58085714, 0.58257143, 0.57914286, 0.58057143,
       0.586     , 0.58457143, 0.57971429, 0.58028571, 0.57314286,
       0.59142857, 0.59171429, 0.59142857, 0.59828571, 0.59771429,
       0.59942857, 0.60142857, 0.60171429, 0.61057143, 0.61      ,
       0.61142857, 0.61857143, 0.618     ])

In [60]:
# 최적의 파라미터
model_gs.best_params_

{'metric': 'manhattan', 'n_neighbors': 27}

In [61]:
# 그때의 성능
model_gs.best_score_

0.6185714285714285

In [62]:
# best 모델로 예측 및 평가
pred = model_gs.predict(x_val_s)
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.59      0.72      0.65       738
           1       0.65      0.51      0.57       762

    accuracy                           0.61      1500
   macro avg       0.62      0.61      0.61      1500
weighted avg       0.62      0.61      0.61      1500



### (5) 실습 : Grid Search

* decision tree로 튜닝을 시도해 봅시다.
    * max_depth : 1~10
    * min_samples_leaf : 10 ~ 100

#### ① 값의 범위를 지정한다.

In [64]:
params = {'max_depth' : range(1, 11), 'min_samples_leaf' : range(10, 101)}
params

{'max_depth': range(1, 11), 'min_samples_leaf': range(10, 101)}

#### ② 모델 선언

In [66]:
model = DecisionTreeClassifier()

model_gs = GridSearchCV(model, params, cv=5) # verbose=2 각각 실행 정보 출력 

#### ③ 모델링

In [70]:
# 학습 : model이 아니라 model_rs
model_gs.fit(x_train_s, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(1, 11),
                         'min_samples_leaf': range(10, 101)})

In [ ]:
# 튜닝 결과
model_gs.cv_results_

In [ ]:
model_gs.cv_results_['params']

In [ ]:
model_gs.cv_results_['mean_test_score']

In [72]:
# 최적의 파라미터
model_gs.best_params_

{'max_depth': 7, 'min_samples_leaf': 28}

In [73]:
# 그때의 성능
model_gs.best_score_

0.7017142857142857

In [74]:
# best 모델로 예측 및 평가
pred = model_gs.predict(x_val_s)
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.66      0.70      0.68       738
           1       0.69      0.66      0.68       762

    accuracy                           0.68      1500
   macro avg       0.68      0.68      0.68      1500
weighted avg       0.68      0.68      0.68      1500

